### **Importing Libraries**

In [77]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import re

### **Reading dataset from drive**

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
df = pd.read_csv('/content/drive/MyDrive/PI-DAintern/Week 3/chipotle.tsv', sep='\t')
df

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


### **Identifying Number of rows and columns**

In [80]:
df.shape

(4622, 5)

In [81]:
df.columns

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')

### **Identifying Missing Values**

In [82]:
missing_values = df.isnull().sum()
missing_values

order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

### **Filling Missing Values**

In [83]:
df['choice_description'].fillna('None', inplace = True)
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,None,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,None,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


### **Data Types**

In [84]:
df.dtypes

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

### **Converting the item price to float data type and removing $**

In [85]:
df['item_price'] = df['item_price'].str.replace('$','').astype(float)
df['item_price']

<ipython-input-85-cc1f095060fe>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['item_price'] = df['item_price'].str.replace('$','').astype(float)


0        2.39
1        3.39
2        3.39
3        2.39
4       16.98
        ...  
4617    11.75
4618    11.75
4619    11.25
4620     8.75
4621     8.75
Name: item_price, Length: 4622, dtype: float64

### **Identify duplicates**

In [86]:
#identify duplicates

df.duplicated().sum()

59

### **Now drop the duplicates**

In [87]:
df.drop_duplicates(inplace = True)

### **After removing duplicates remaining rows left**

In [88]:
df.shape

(4563, 5)

### **Quantity and Item Price**

In [89]:
#Examining the Quantity and Item Price columns. To see any inconsistencies or anomalies that need corrected

df[['quantity', 'item_price']].describe()

,quantity,item_price
count,4563.000000,4563.000000
mean,1.076704,7.490083
std,0.412739,4.244155
min,1.000000,1.090000
25%,1.000000,3.750000
50%,1.000000,8.750000
75%,1.000000,9.250000
max,15.000000,44.250000


The data seems consistent

### **Handling Choice Description**

In [90]:
df['choice_description'].head(10)

0                                                 None
1                                         [Clementine]
2                                              [Apple]
3                                                 None
4    [Tomatillo-Red Chili Salsa (Hot), [Black Beans...
5    [Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...
6                                                 None
7    [Tomatillo Red Chili Salsa, [Fajita Vegetables...
8    [Tomatillo Green Chili Salsa, [Pinto Beans, Ch...
9    [Fresh Tomato Salsa, [Rice, Black Beans, Pinto...
Name: choice_description, dtype: object

In [91]:
#Inorder to easily handle multiple choice descrption let's split choice description as choice desciprion1,2...
split_columns = df['choice_description'].str.split(',', expand = True)
split_columns.columns = ['Choice_Description_1', 'Choice_Description_2',
                         'Choice_Description_3', 'Choice_Description_4',
                         'Choice_Description_5','Choice_Description_6',
                         'Choice_Description_7', 'Choice_Description_8',
                         'Choice_Description_9', 'Choice_Description_10']
df = pd.concat([df, split_columns], axis = 1)
df.drop('choice_description', axis = 1, inplace = True)
df.head()

,order_id,quantity,item_name,item_price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,[Clementine],None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,[Apple],None,None,None,None,None,None,None,None,None
3,1,1,Chips and Tomatillo-Green Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,[Tomatillo-Red Chili Salsa (Hot),[Black Beans,Rice,Cheese,Sour Cream]],None,None,None,None,None


### **Handling Special Characters**

In [92]:
columns_to_check = ['item_name','Choice_Description_1','Choice_Description_2',
                    'Choice_Description_3','Choice_Description_4',
                    'Choice_Description_5','Choice_Description_6',
                    'Choice_Description_7','Choice_Description_8',
                    'Choice_Description_9','Choice_Description_10']

In [93]:
#identify special characters
special_chars = re.compile(r'[^a-zA-Z0-9\s]')

In [94]:
df[columns_to_check] = df[columns_to_check].applymap(lambda x: special_chars.sub('',x)if isinstance(x, str) else x)
df.head()

,order_id,quantity,item_name,item_price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,Apple,None,None,None,None,None,None,None,None,None
3,1,1,Chips and TomatilloGreen Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None


In [95]:
#cheching for special characters in specified columns using str.contains()
special_chars_check = df[columns_to_check].apply(lambda col: col.str.contains(special_chars))
special_chars_check
df.head()

,order_id,quantity,item_name,item_price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,2.39,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.39,Apple,None,None,None,None,None,None,None,None,None
3,1,1,Chips and TomatilloGreen Chili Salsa,2.39,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,16.98,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None


### **Order ID Integrity**

In [96]:
# Check for unique Order Ids

unique_orders = df['order_id'].unique()

In [97]:
# Check for irregularities in order ids

if len(unique_orders) == df.shape[0]:
    print("Order IDs are unique and consistent.")
else:
    print("Irregularities or duplicate Order IDs.")

Irregularities or duplicate Order IDs.


In [98]:
# Check for missing values

missing_values = df['order_id'].isnull().sum()
if missing_values > 0:
    print(f"There are {missing_values} missing IDs.")
else:
    print("No missing IDs.")

No missing IDs.


In [99]:
# Check for irregularities in Order ids

order_id_counts = df['order_id'].value_counts()
print("Counts of Order ID:")
order_id_counts

Counts of Order ID:


926     21
1483    14
1786    11
759     11
691     11
        ..
94       1
1374     1
837      1
831      1
528      1
Name: order_id, Length: 1834, dtype: int64

In [100]:
# Checking for Integrity in oder ids

df[df['order_id'] == 759]

,order_id,quantity,item_name,item_price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
1872,759,1,Chicken Bowl,11.25,Fresh Tomato Salsa,Rice,Black Beans,Pinto Beans,Guacamole,Lettuce,None,None,None,None
1873,759,1,Carnitas Bowl,9.25,Fresh Tomato Salsa,Rice,Pinto Beans,Cheese,Lettuce,None,None,None,None,None
1874,759,1,Chicken Soft Tacos,8.75,Fresh Tomato Salsa,Cheese,Lettuce,None,None,None,None,None,None,None
1875,759,1,Veggie Bowl,11.25,Roasted Chili Corn Salsa,Rice,Black Beans,Cheese,Sour Cream,Guacamole,Lettuce,None,None,None
1876,759,1,Chicken Bowl,8.75,Tomatillo Green Chili Salsa,Rice,Black Beans,Cheese,Sour Cream,Lettuce,None,None,None,None
1877,759,1,Chicken Bowl,11.25,Fresh Tomato Salsa,Fajita Vegetables,Rice,Guacamole,Lettuce,None,None,None,None,None
1878,759,2,Canned Soft Drink,2.50,Coke,None,None,None,None,None,None,None,None,None
1879,759,2,Canned Soft Drink,2.50,Diet Coke,None,None,None,None,None,None,None,None,None
1880,759,4,Bottled Water,6.00,None,None,None,None,None,None,None,None,None,None
1881,759,2,Chips and Guacamole,8.90,None,None,None,None,None,None,None,None,None,None


### **Standaerdizing Item Name**

In [101]:
df['item_name']
unique_item_names = df['item_name'].unique()
unique_item_names

array(['Chips and Fresh Tomato Salsa', 'Izze', 'Nantucket Nectar',
       'Chips and TomatilloGreen Chili Salsa', 'Chicken Bowl',
       'Side of Chips', 'Steak Burrito', 'Steak Soft Tacos',
       'Chips and Guacamole', 'Chicken Crispy Tacos',
       'Chicken Soft Tacos', 'Chicken Burrito', 'Canned Soda',
       'Barbacoa Burrito', 'Carnitas Burrito', 'Carnitas Bowl',
       'Bottled Water', 'Chips and Tomatillo Green Chili Salsa',
       'Barbacoa Bowl', 'Chips', 'Chicken Salad Bowl', 'Steak Bowl',
       'Barbacoa Soft Tacos', 'Veggie Burrito', 'Veggie Bowl',
       'Steak Crispy Tacos', 'Chips and Tomatillo Red Chili Salsa',
       'Barbacoa Crispy Tacos', 'Veggie Salad Bowl',
       'Chips and Roasted ChiliCorn Salsa',
       'Chips and Roasted Chili Corn Salsa', 'Carnitas Soft Tacos',
       'Chicken Salad', 'Canned Soft Drink', 'Steak Salad Bowl',
       '6 Pack Soft Drink', 'Chips and TomatilloRed Chili Salsa', 'Bowl',
       'Burrito', 'Crispy Tacos', 'Carnitas Crispy Tacos', 

In [102]:
pattern = r'[^a-zA-Z0-9\s]'
df['item_name']

0               Chips and Fresh Tomato Salsa
1                                       Izze
2                           Nantucket Nectar
3       Chips and TomatilloGreen Chili Salsa
4                               Chicken Bowl
                        ...                 
4617                           Steak Burrito
4618                           Steak Burrito
4619                      Chicken Salad Bowl
4620                      Chicken Salad Bowl
4621                      Chicken Salad Bowl
Name: item_name, Length: 4563, dtype: object

In [103]:
#removing special characters from 'Item_Name'

df['item_name']=df['item_name'].str.replace(pattern, '', regex=True)
df['item_name']

0               Chips and Fresh Tomato Salsa
1                                       Izze
2                           Nantucket Nectar
3       Chips and TomatilloGreen Chili Salsa
4                               Chicken Bowl
                        ...                 
4617                           Steak Burrito
4618                           Steak Burrito
4619                      Chicken Salad Bowl
4620                      Chicken Salad Bowl
4621                      Chicken Salad Bowl
Name: item_name, Length: 4563, dtype: object

### **Quantity and Price Relationships**

In [104]:
df[['quantity','item_price']]

,quantity,item_price
0,1,2.39
1,1,3.39
2,1,3.39
3,1,2.39
4,2,16.98
...,...,...
4617,1,11.75
4618,1,11.75
4619,1,11.25
4620,1,8.75


In [105]:
df[['quantity', 'item_price']].describe()

,quantity,item_price
count,4563.000000,4563.000000
mean,1.076704,7.490083
std,0.412739,4.244155
min,1.000000,1.090000
25%,1.000000,3.750000
50%,1.000000,8.750000
75%,1.000000,9.250000
max,15.000000,44.250000


In [106]:
item_costs = df.groupby('item_name')['item_price'].mean().to_dict()

In [107]:
expected_price = df['quantity'] * df['item_name'].map(item_costs)
inconsistent_prices = abs(expected_price - df['item_price']) > 0.01
df.loc[inconsistent_prices, 'item_price'] = expected_price[inconsistent_prices]

In [108]:
df['item_price']=df['item_price'].round(2)
df['item_price']=df['item_price'].round(2)

In [109]:
#top 5 items by Quantity and Price
df.head()

,order_id,quantity,item_name,item_price,Choice_Description_1,Choice_Description_2,Choice_Description_3,Choice_Description_4,Choice_Description_5,Choice_Description_6,Choice_Description_7,Choice_Description_8,Choice_Description_9,Choice_Description_10
0,1,1,Chips and Fresh Tomato Salsa,3.29,None,None,None,None,None,None,None,None,None,None
1,1,1,Izze,3.39,Clementine,None,None,None,None,None,None,None,None,None
2,1,1,Nantucket Nectar,3.64,Apple,None,None,None,None,None,None,None,None,None
3,1,1,Chips and TomatilloGreen Chili Salsa,2.54,None,None,None,None,None,None,None,None,None,None
4,2,2,Chicken Bowl,20.25,TomatilloRed Chili Salsa Hot,Black Beans,Rice,Cheese,Sour Cream,None,None,None,None,None


### **Data Integrity Check**

In [110]:
#data integrity check by ensuring that quantities and prices align with the corresponding items and descriptions.

df.groupby('item_name')['quantity'].sum().sort_values(ascending = False).head(5)
df.groupby('item_name')['item_price'].sum().sort_values(ascending = False).head(5)

item_name
Chicken Bowl           7617.43
Chicken Burrito        5892.56
Steak Burrito          4006.34
Steak Bowl             2358.40
Chips and Guacamole    2304.36
Name: item_price, dtype: float64